## ライブラリのインポート

In [1]:
from collections import Counter

import pandas as pd
from sudachipy import dictionary, tokenizer
from tqdm import tqdm

## データの読み込み

In [2]:
# プロジェクトデータの読み込み
project_df = pd.read_csv("../data/raw/project-true.csv")
print(f"プロジェクトデータ: {len(project_df)}件")
print(project_df.head())

# ユーザー職歴データの読み込み
user_df = pd.read_csv(
    "../data/raw/user_work_histories.csv",
    usecols=["user_id", "project_name", "description", "role"],
)
print(f"\nユーザー職歴データ: {len(user_df)}件")
print(user_df.head())

プロジェクトデータ: 1000件
   id          name                                        description  \
0   1    旅行予約システム設計  当プロジェクトでは運用保守、テスト、デプロイ、ドキュメント作成、自動テスト導入やインフラ構築...   
1   2    予約管理システム設計  現プロジェクトにおいてデータベース設計やデプロイなどに取り組んでいる。\n[実施内容]\nデ...   
2   3  ブロックチェーン開発実装  要件定義、要件分析、パフォーマンス改善、自動テスト導入、クラウド移行やセキュリティ強化などの...   
3   4    動画配信サービス改善  技術選定、UI/UX設計、セキュリティ強化、実装、デプロイやデータベース設計などの開発に取り...   
4   5    物流管理システム改善             データベース設計、運用保守やチームマネジメントなどの開発に取り組んでいます。   

   start_date end_date  
0  2024-04-09      NaN  
1  2024-12-22      NaN  
2  2025-05-20      NaN  
3  2024-11-24      NaN  
4  2024-02-22      NaN  

ユーザー職歴データ: 911件
   user_id project_name                                        description  \
0        1     スマホアプリ開発    SNSアプリ開発\n[主な担当]\nドキュメント作成\nテスト実施\nインフラ構築\n技術検証   
1        1   医療情報システム開発  オンプレミス環境からAWSへのクラウド移行を主導しました。 新技術の調査やPoCを行い、アー...   
2        1   データ分析基盤の構築  アジャイル手法で開発を進め、定期的なスプリントレビューで改善を行いました。 自動テストの導入...   
3        1   動画配信サービス構築  セキュリティ強化プロジェクト\n[主な担当]\n要件定義\nパフォー

## 名詞抽出関数の定義

In [3]:
def extract_nouns(text: str):
    """
    テキストから名詞を抽出する関数
    """
    if pd.isna(text):
        return []

    # full辞書を使用して辞書オブジェクトを生成
    tokenizer_obj = dictionary.Dictionary(dict="full").create()
    # 形態素解析の粒度設定(C: 最も粗い)
    mode = tokenizer.Tokenizer.SplitMode.C

    # 形態素解析を実行
    morphemes = tokenizer_obj.tokenize(str(text), mode)

    nouns = []
    for m in morphemes:
        # 品詞情報を取得
        pos = m.part_of_speech()[0]
        if pos == "名詞":
            nouns.append(m.surface())
    return nouns


# 動作確認
sample_text = "AWSを使ったWebアプリケーション開発プロジェクト"
print(f"サンプルテキスト: {sample_text}")
print(f"抽出された名詞: {extract_nouns(sample_text)}")

サンプルテキスト: AWSを使ったWebアプリケーション開発プロジェクト
抽出された名詞: ['AWS', 'Web', 'アプリケーション', '開発', 'プロジェクト']


## Word2Vec用のコーパス作成

In [4]:
def create_corpus_from_df(df, columns, desc="Processing"):
    """
    DataFrameから指定列のテキストを抽出し、名詞のみのリストのリストを作成

    Parameters:
    -----------
    df : pd.DataFrame
        対象のDataFrame
    columns : list
        テキスト抽出対象の列名リスト
    desc : str
        進行状況バーの説明文

    Returns:
    --------
    corpus : list of list
        各行の名詞リスト
    """
    corpus = []

    for _, row in tqdm(df.iterrows(), total=len(df), desc=desc):
        # 各行の指定列から名詞を抽出
        row_nouns = []
        for col in columns:
            if col in df.columns and pd.notna(row[col]):
                nouns = extract_nouns(str(row[col]))
                row_nouns.extend(nouns)

        if row_nouns:  # 空でない場合のみ追加
            corpus.append(row_nouns)

    return corpus


# プロジェクトデータのコーパスを作成
print("プロジェクトデータの処理中...")
project_columns = ["name", "description"]
project_corpus = create_corpus_from_df(
    project_df, project_columns, desc="プロジェクトデータ"
)

# ユーザー職歴のコーパスを作成
print("\nユーザー職歴データの処理中...")
user_columns = ["project_name", "description", "role"]
user_corpus = create_corpus_from_df(user_df, user_columns, desc="ユーザー職歴データ")

# 両方を結合
corpus = project_corpus + user_corpus

print(f"\n✓ プロジェクトデータコーパス: {len(project_corpus)}件")
print(f"✓ ユーザー職歴コーパス: {len(user_corpus)}件")
print(f"✓ 結合コーパス: {len(corpus)}件")

print("\n【プロジェクトデータ】最初の2件のサンプル:")
for i, doc in enumerate(project_corpus[:2]):
    print(f"{i + 1}: {doc[:15]}...")

print("\n【ユーザー職歴データ】最初の2件のサンプル:")
for i, doc in enumerate(user_corpus[:2]):
    print(f"{i + 1}: {doc[:15]}...")

プロジェクトデータの処理中...


プロジェクトデータ: 100%|██████████| 1000/1000 [00:28<00:00, 35.67it/s]



ユーザー職歴データの処理中...


ユーザー職歴データ: 100%|██████████| 911/911 [00:37<00:00, 24.21it/s]


✓ プロジェクトデータコーパス: 1000件
✓ ユーザー職歴コーパス: 911件
✓ 結合コーパス: 1911件

【プロジェクトデータ】最初の2件のサンプル:
1: ['旅行', '予約', 'システム設計', 'プロジェクト', '運用', '保守', 'テスト', 'デプロイ', 'ドキュメント', '作成', '自動', 'テスト', '導入', 'インフラ', '構築']...
2: ['予約', '管理', 'システム設計', 'プロジェクト', 'データベース設計', 'デプロイ', '実施', '内容', 'データベース設計', 'デプロイ', '実装', '自動', 'テスト', '導入', '運用']...

【ユーザー職歴データ】最初の2件のサンプル:
1: ['スマホアプリ', '開発', 'SNS', 'アプリ', '開発', '担当', 'ドキュメント', '作成', 'テスト', '実施', 'インフラ', '構築', '技術', '検証', 'プログラマー']...
2: ['医療情報システム', '開発', 'オンプレミス', '環境', 'AWS', 'クラウド', '移行', '主導', '新技術', '調査', 'PoC', 'アーキテクチャ', '改善', '提案', 'バックエンド']...


## コーパスをテキストファイルに保存

In [5]:
# コーパスをテキストファイルに保存
save_path_txt = "../data/processed/corpus-true.txt"
with open(save_path_txt, "w", encoding="utf-8") as f:
    for doc in corpus:
        f.write(" ".join(doc) + "\n")
print(f"コーパスをテキストファイルに保存しました: {save_path_txt}")

コーパスをテキストファイルに保存しました: ../data/processed/corpus-true.txt


## コーパスの統計情報

In [6]:
# 全単語の頻度を集計
all_words = [word for doc in corpus for word in doc]
word_freq = Counter(all_words)

print(f"総単語数: {len(all_words):,}")
print(f"ユニーク単語数: {len(word_freq):,}")

print("\n頻出単語 上位30:")
for i, (word, count) in enumerate(word_freq.most_common(30), 1):
    print(f"{i:2d}. {word}: {count:,}回")

総単語数: 32,852
ユニーク単語数: 193

頻出単語 上位30:
 1. 開発: 1,116回
 2. 構築: 1,026回
 3. 設計: 993回
 4. プロジェクト: 892回
 5. テスト: 870回
 6. 担当: 805回
 7. 改善: 739回
 8. 実装: 634回
 9. レビュー: 525回
10. クラウド: 510回
11. 導入: 498回
12. チーム: 492回
13. システム設計: 415回
14. セキュリティ: 415回
15. 移行: 413回
16. UI: 408回
17. UX: 408回
18. 要件: 404回
19. 技術: 398回
20. パフォーマンス: 388回
21. マネジメント: 388回
22. リファクタリング: 381回
23. データベース設計: 369回
24. 要件定義: 369回
25. アーキテクチャ: 341回
26. アプリ: 333回
27. 運用: 327回
28. 強化: 327回
29. 保守: 321回
30. システム: 307回
